In [1]:
!pip install streamlit -q
!pip install streamlit-lottie
!pip install Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 38.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 7.3 MB/s eta 0:00:00


In [8]:
pip install shap

In [56]:
!streamlit run app.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.148.253.209:8501

npx: installed 22 in 2.143s
your url is: https://fresh-dots-smash.loca.lt
2024-05-27 16:27:20.467858: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-27 16:27:20.467923: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-27 16:27:20.469593: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-27 16:27:20.479788: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimiz

In [52]:
%%writefile app.py
import streamlit as st
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt
from datetime import datetime


st.title("Anomaly Detection System")
st.image("/content/logo.png",  width=200)


metric_columns = ['bandwidth','throughput', 'congestion', 'packet_loss', 'latency', 'jitter']
def detect_anomalies_iqr(data, metric, threshold=1.2):
    Q1 = data[metric].quantile(0.25)
    Q3 = data[metric].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - threshold * IQR
    upper_bound = Q3 + threshold * IQR
    return (data[metric] < lower_bound) | (data[metric] > upper_bound)

#@st.cache
@st.cache_data
#@st.cache_data()
def load_data():
    df = pd.read_csv('/content/dataset.csv')
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    return df

df = load_data()

option = st.selectbox("Choose an option:", ("Detection", "Prediction"),index=None,placeholder="Select method...",)
st.write("You selected:",option)

if option == "Detection":

  st.subheader("Insert new network data")
  new_data = {}
  for metric in metric_columns:
      new_value = st.text_input(f'{metric}:')
      new_data[metric] = float(new_value) if new_value else None

  if st.button('Add data',type="primary"):
      with st.expander("Anomaly threshold"):
          last_timestamp = df['timestamp'].max()
          new_data['timestamp'] = last_timestamp + pd.Timedelta(seconds=180)
          new_data_df = pd.DataFrame([new_data], index=[1001])
          df = pd.concat([df, new_data_df])

          for metric in metric_columns:
                df[f'anomaly_{metric}'] = 0

          for metric in metric_columns:
                df.loc[detect_anomalies_iqr(df, metric), f'anomaly_{metric}'] = 1



                fig, ax = plt.subplots(figsize=(12, 6))
                ax.plot(df['timestamp'], df[metric], label=metric)
                ax.scatter(df[df[f'anomaly_{metric}'] == 1]['timestamp'], df[df[f'anomaly_{metric}'] == 1][metric], color='red', label='Anomaly')
                ax.set_title(f'{metric} with Anomalies (IQR)')
                ax.set_ylabel(f'{metric} Values')
                ax.set_xlabel('Timestamp')
                ax.legend()
                st.pyplot(fig)


          df['anomaly'] = (df[[f'anomaly_{metric}' for metric in metric_columns]].sum(axis=1) >= 2).astype(int)

          df['-'] = df['anomaly'].apply(lambda x: ' ' if x == 0 else '❌')


      with st.expander('Labeled data'):
        st.write(df)
        percentage_anomalies = (df['anomaly'].sum() / len(df)) * 100
        st.write("Anomaly percentage:", percentage_anomalies, "%")


      with st.expander("Logistic Regression Classification"):
        import pandas as pd
        from imblearn.over_sampling import SMOTE
        from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
        from sklearn.preprocessing import StandardScaler
        from sklearn.linear_model import LogisticRegression
        from sklearn.ensemble import RandomForestClassifier
        from sklearn.cluster import KMeans
        from sklearn.metrics import classification_report, confusion_matrix
        import matplotlib.pyplot as plt


        df = pd.read_csv('dataset_label_finalll.csv')


        df.index = df['timestamp']

        metric_columns = ['bandwidth', 'throughput', 'congestion', 'packet_loss', 'latency', 'jitter', 'anomaly']
        df = df[metric_columns]


        X = df.drop('anomaly', axis=1)
        y = df['anomaly']
        smote = SMOTE(random_state=42)
        X, y = smote.fit_resample(X, y)

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)


        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


        logistic_regression_model = LogisticRegression()

        logistic_regression_cv_scores = cross_val_score(logistic_regression_model, X_train_scaled, y_train, cv=kf, scoring='accuracy')

        st.write("Logistic Regression Accuracy:", logistic_regression_cv_scores.mean())


        logistic_regression_model.fit(X_train_scaled, y_train)
        X_test2 = scaler.transform(df.drop('anomaly', axis=1))
        predic = scaler.transform(new_data_df.drop('timestamp', axis=1))

        logistic_regression_predictions = logistic_regression_model.predict(X_test_scaled)
        st.write("\nLogistic Regression Confusion Matrix:\n", confusion_matrix(y_test, logistic_regression_predictions))

        st.text(classification_report(y_test, logistic_regression_predictions))
        logistic_regression_predictions2 = logistic_regression_model.predict(X_test2)
        logistic_regression_predic = logistic_regression_model.predict(predic)
        logistic_regression_anomaly_percentage = (logistic_regression_predictions2.sum() / len(logistic_regression_predictions2)) * 100
        st.write("Anomaly percentage detected with Logistic Regression:", logistic_regression_anomaly_percentage)
        if logistic_regression_predic == 1:
            st.write("The model DETECTED an anomaly in the new input data!")
        else:
            st.write("The model did NOT DETECT any anomaly in the new input data.")
      with st.expander("Random Forest Classification"):

        random_forest_model = RandomForestClassifier()

        random_forest_cv_scores = cross_val_score(random_forest_model, X_train, y_train, cv=kf, scoring='accuracy')

        st.write("Random Forest Accuracy:", random_forest_cv_scores.mean())


        random_forest_model.fit(X_train, y_train)


        random_forest_predictions = random_forest_model.predict(X_test)
        st.write("\nRandom Forest Confusion Matrix:\n", confusion_matrix(y_test, random_forest_predictions))

        st.text(classification_report(y_test, random_forest_predictions))
        X_test2 = (df.drop('anomaly', axis=1))
        predic=(new_data_df.drop('timestamp', axis=1))

        random_forest_predictions2 = random_forest_model.predict(X_test2)
        random_forest_predic = random_forest_model.predict(predic)
        if random_forest_predic == 1:
            st.write("The model DETECTED an anomaly in the new input data!")
        else:
            st.write("The model did NOT DETECT any anomaly in the new input data.")
        random_forest_anomaly_percentage = (random_forest_predictions2.sum() / len(random_forest_predictions2)) * 100
        st.write("Anomaly percentage detected with Random Forest:", random_forest_anomaly_percentage)


      with st.expander("SVM Classification"):
        import pandas as pd
        from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
        from sklearn.svm import SVC
        from sklearn.metrics import classification_report, confusion_matrix
        from sklearn.preprocessing import StandardScaler
        from imblearn.over_sampling import SMOTE
        import numpy as np


        df = pd.read_csv('dataset_label_finalll.csv')


        metric_columns = ['bandwidth', 'throughput', 'congestion', 'packet_loss', 'latency', 'jitter', 'anomaly']
        df = df[metric_columns]


        X = df.drop('anomaly', axis=1)
        y = df['anomaly']


        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)


        smote = SMOTE(random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X_scaled, y)


        kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


        svm_model = SVC(kernel='rbf', C=1, gamma='auto')


        cv_scores = cross_val_score(svm_model, X_resampled, y_resampled, cv=kf, scoring='accuracy')


        st.write("SVM Accuracy:", cv_scores.mean())


        X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)


        svm_model.fit(X_train, y_train)


        svm_predictions = svm_model.predict(X_test)
        columns = X.columns
        predic=scaler.fit_transform(new_data_df.drop('timestamp', axis=1))

        X_scaled_df = pd.DataFrame(X_scaled, columns=columns)

        svm_predictions2 = svm_model.predict(X_scaled_df)
        svm_predic=svm_model.predict(predic)

        st.write("\nSVM Confusion Matrix:\n", confusion_matrix(y_test, svm_predictions))

        st.text(classification_report(y_test, svm_predictions))
        svm_anomaly_percentage = (np.sum(svm_predictions2) / len(svm_predictions2)) * 100
        st.write("Anomaly percentage detected with SVM:", svm_anomaly_percentage)
        if svm_predic == 1:
            st.write("The model DETECTED an anomaly in the new input data!")
        else:
            st.write("The model did NOT DETECT any anomaly in the new input data.")


elif option == "Prediction":
  with st.expander("LSTM"):
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import MinMaxScaler
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM, Dense
    from sklearn.model_selection import train_test_split


    df = pd.read_csv('dataset.csv')


    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df['throughput'].values.reshape(-1, 1))


    def create_dataset(data, time_steps=1):
        X, y = [], []
        for i in range(len(data) - time_steps):
            X.append(data[i:(i + time_steps), 0])
            y.append(data[i + time_steps, 0])
        return np.array(X), np.array(y)


    time_steps = 10


    X, y = create_dataset(scaled_data, time_steps)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        LSTM(units=50),
        Dense(units=1)
    ])


    model.compile(optimizer='adam', loss='mean_squared_error')


    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


    loss = model.evaluate(X_test, y_test)
    st.write("Loss in the test data:", loss)
    predictions = model.predict(X_test)

    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))
    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_test, predictions)

    st.write("Mean square error in test data:", mse)


    fig, ax = plt.subplots(figsize=(14, 7))
    ax.plot(y_test, label='True')
    ax.plot(predictions, label='Predicted')
    ax.set_title("Predictions vs. true values")
    ax.set_xlabel("Index")
    ax.set_ylabel("Throughput")
    ax.legend()


    st.pyplot(fig)



    df = pd.read_csv('dataset.csv')


    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df['congestion'].values.reshape(-1, 1))


    def create_dataset(data, time_steps=1):
        X, y = [], []
        for i in range(len(data) - time_steps):
            X.append(data[i:(i + time_steps), 0])
            y.append(data[i + time_steps, 0])
        return np.array(X), np.array(y)


    time_steps = 10


    X, y = create_dataset(scaled_data, time_steps)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        LSTM(units=50),
        Dense(units=1)
    ])


    model.compile(optimizer='adam', loss='mean_squared_error')


    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


    loss = model.evaluate(X_test, y_test)
    st.write("Loss in the test data:", loss)


    predictions = model.predict(X_test)


    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))


    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_test, predictions)
    st.write("Mean square error in test data:", mse)


    fig, ax = plt.subplots(figsize=(14, 7))
    ax.plot(y_test, label='True')
    ax.plot(predictions, label='Predicted')
    ax.set_title("Predictions vs. true values")
    ax.set_xlabel("Index")
    ax.set_ylabel("Congestion")
    ax.legend()


    st.pyplot(fig)



    df = pd.read_csv('dataset.csv')


    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df['packet_loss'].values.reshape(-1, 1))


    def create_dataset(data, time_steps=1):
        X, y = [], []
        for i in range(len(data) - time_steps):
            X.append(data[i:(i + time_steps), 0])
            y.append(data[i + time_steps, 0])
        return np.array(X), np.array(y)


    time_steps = 10


    X, y = create_dataset(scaled_data, time_steps)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        LSTM(units=50),
        Dense(units=1)
    ])


    model.compile(optimizer='adam', loss='mean_squared_error')


    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


    loss = model.evaluate(X_test, y_test)
    st.write("Loss in the test data:", loss)


    predictions = model.predict(X_test)


    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))


    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_test, predictions)
    st.write("Mean square error in test data:", mse)


    import matplotlib.pyplot as plt



    fig, ax = plt.subplots(figsize=(14, 7))
    ax.plot(y_test, label='True')
    ax.plot(predictions, label='Predicted')
    ax.set_title("Predictions vs. true values")
    ax.set_xlabel("Index")
    ax.set_ylabel("Packet loss")
    ax.legend()


    st.pyplot(fig)



    df = pd.read_csv('dataset.csv')


    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df['jitter'].values.reshape(-1, 1))


    def create_dataset(data, time_steps=1):
        X, y = [], []
        for i in range(len(data) - time_steps):
            X.append(data[i:(i + time_steps), 0])
            y.append(data[i + time_steps, 0])
        return np.array(X), np.array(y)


    time_steps = 10


    X, y = create_dataset(scaled_data, time_steps)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        LSTM(units=50),
        Dense(units=1)
    ])


    model.compile(optimizer='adam', loss='mean_squared_error')


    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


    loss = model.evaluate(X_test, y_test)
    st.write("Loss in the test data:", loss)



    predictions = model.predict(X_test)


    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))


    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_test, predictions)
    st.write("Mean square error in test data:", mse)


    fig, ax = plt.subplots(figsize=(14, 7))
    ax.plot(y_test, label='True')
    ax.plot(predictions, label='Predicted')
    ax.set_title("Predictions vs. true values")
    ax.set_xlabel("Index")
    ax.set_ylabel("Jitter")
    ax.legend()
    st.pyplot(fig)


    df = pd.read_csv('dataset.csv')


    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df['latency'].values.reshape(-1, 1))


    def create_dataset(data, time_steps=1):
        X, y = [], []
        for i in range(len(data) - time_steps):
            X.append(data[i:(i + time_steps), 0])
            y.append(data[i + time_steps, 0])
        return np.array(X), np.array(y)


    time_steps = 10


    X, y = create_dataset(scaled_data, time_steps)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


    model = Sequential([
        LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)),
        LSTM(units=50),
        Dense(units=1)
    ])


    model.compile(optimizer='adam', loss='mean_squared_error')


    model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))


    loss = model.evaluate(X_test, y_test)
    st.write("Loss in the test data:", loss)


    predictions = model.predict(X_test)


    predictions = scaler.inverse_transform(predictions)
    y_test = scaler.inverse_transform(y_test.reshape(-1, 1))


    from sklearn.metrics import mean_squared_error
    mse = mean_squared_error(y_test, predictions)
    st.write("Mean square error in test data:", mse)


    import matplotlib.pyplot as plt


    fig, ax = plt.subplots(figsize=(14, 7))
    ax.plot(y_test, label='True')
    ax.plot(predictions, label='Predicted')
    ax.set_title("Predictions vs. true values")
    ax.set_xlabel("Index")
    ax.set_ylabel("Latency")
    ax.legend()
    st.pyplot(fig)

Overwriting app.py
